# Best Practice

## 0. DB

In [32]:
from dotenv import load_dotenv

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [33]:
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

samsung_vision_2025_path = '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf'
docs_2025 = PyPDFLoader(samsung_vision_2025_path).load()

print(len(docs_2025))

87


In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks_2025 = text_splitter.split_documents(docs_2025)

print(len(chunks_2025))

237


In [ ]:
collection_name = 'best_data'
db_path = '../vectorstore/best_db'

# db save
db_2025 = Chroma.from_documents(
    documents=chunks_2025, 
    collection_name=collection_name, 
    persist_directory=db_path, 
    embedding=embedding
)

## 1. RAG

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# db load
vector_db = Chroma(
    persist_directory = db_path,
    collection_name = collection_name,
    embedding_function = embedding
)

In [38]:
sim_retriever = vector_db.as_retriever(
    search_type = 'similarity',
    search_kwargs = {
        'k': 30
    }
)

In [ ]:
# reranker

from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

hf_ce = HuggingFaceCrossEncoder(
    model_name = 'cross-encoder/ms-marco-MiniLM-L6-v2',
    model_kwargs = {
        'device' : 'cuda',
        'max_length' : 512
    }
)

reranker = CrossEncoderReranker(
    model = hf_ce,
    top_n = 10
)

In [40]:
from langchain.retrievers import ContextualCompressionRetriever

com_retriever = ContextualCompressionRetriever(
    base_retriever = sim_retriever,
    base_compressor = reranker
)

In [ ]:
# reorder

from langchain_community.document_transformers import LongContextReorder

reorder  = LongContextReorder()

In [42]:
def format_docs(docs):
    
    result = []
    
    for doc in docs:
        result.append(doc.page_content.strip())
    
    return '\n\n---\n\n'.join(result)


## 2. CHAIN

In [43]:
prompt = ChatPromptTemplate.from_messages([
    ('system', """
    주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라.
    # Context
    {context}
    """),
    ('human', '{question}')
])

model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0
)

output_parser = StrOutputParser()

llm_chain = prompt | model | output_parser

## 3. TOTAL(RAG + CHAIN)

In [44]:
# 중간 단계의 가변성을 위해서 람다를 적극적으로 활용
total_chain1 = (
    {
    'docs': RunnableLambda(lambda x: com_retriever.invoke(x['question'])),
    'question': RunnablePassthrough()    
    }
    | RunnableLambda(lambda x: {
        'context': format_docs(reorder.transform_documents(x['docs'])),  
        'question': x['question']
    })
    | llm_chain
)

total_chain1.invoke({
    'question' : '삼성전자의 미래 사업 계획'
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래, 기술 리더십으로 재도약의 기반을 다지고 새로운 영역에서 미래 성장동력을 확보해 나갈 계획입니다. 또한, 2025년까지 사회공헌 분야에서 청년 소프트웨어 인재 양성과 자립 준비 청년 지원을 확대하며, 환경 분야에서는 DX부문 2030년, DS부문 2050년 탄소중립 달성을 목표로 재생에너지 전환과 자원순환 극대화에 노력할 예정입니다.'

In [45]:
total_chain2 = (
    {
    'context': RunnableLambda(lambda x: x['question']) | com_retriever | reorder.transform_documents | format_docs,
    'question': RunnablePassthrough()
    } 
    | llm_chain
)

total_chain2.invoke({
    'question' : '삼성전자의 미래 사업 계획'
})

"삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래, 기술 리더십으로 재도약의 기반을 다지고 새로운 영역에서 미래 성장동력을 확보할 계획입니다. 또한, 2025년에는 '삼성 청년SW·AI아카데미' 교육 대상을 마이스터고 졸업생까지 확대하고, '삼성 희망디딤돌' 인천센터를 추가 설립하여 더 많은 청년을 지원할 예정입니다. 환경 분야에서는 DX부문이 2030년 탄소중립 달성을 목표로 재생에너지 전환과 에너지 효율 개선에 집중하며, DS부문은 2050년 탄소중립을 목표로 하고 있습니다."

## 4. TOTAL(+Multi Input)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ('system', """
    주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라.
    # Context
    {context}
    """),
    ('human', '{pro}의 스타일에 맞게 {question}')
])

model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0
)

output_parser = StrOutputParser()

multi_llm_chain = prompt | model | output_parser
multi_llm_chain

ChatPromptTemplate(input_variables=['context', 'pro', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라.\n    # Context\n    {context}\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pro', 'question'], input_types={}, partial_variables={}, template='{pro}의 스타일에 맞게 {question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000206CA731250>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002067624FD90>, root_client=<openai.OpenAI object at 0x0000020682721890>, root_async_client=<openai.AsyncOpenAI object at 0x00000206CA732DD0>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| Str

In [ ]:
total_chain = (
    {
        "context" :  RunnableLambda(lambda x: x['question']) | com_retriever | format_docs,
        "question" : RunnableLambda(lambda x: x['question']),
        "pro" : RunnableLambda(lambda x: x['pro'])
    }
    | multi_llm_chain
)

total_chain.invoke({
    'question' : '삼성의 미래 계획은 어떻게 되나요?',
    "pro" : '초등학생'
})

'삼성전자는 앞으로 모든 청소년이 좋은 교육을 받을 수 있도록 도와줄 거예요. 예를 들어, 컴퓨터와 인공지능을 배우는 특별한 학교를 운영하고, 중학생들이 꿈을 찾고 공부할 수 있게 멘토와 함께 공부하는 프로그램도 있어요. 또, 작은 회사와 새로운 아이디어를 가진 스타트업도 도와서 더 멋진 기술과 제품이 나오도록 할 계획이에요. 이렇게 삼성은 모두가 더 밝은 미래를 만들 수 있도록 힘쓰고 있어요!'